# Point spread function using Tiny Tim/Galfit

Calling tinytim to generate psf is realized by .py script modified from Bryan R. Gillis, the following infomation is required:

0. sensor name

1. detector position

2. wavelengths of the emission lines (redshift)

3. subsampled factor

4. (optional) filter used and focus info



### import modules and defing a few useful little functions

In [1]:
import psf.make_psf as tinytim
import  numpy               as     np
from    astropy.table       import Table
from    astropy.io          import fits
from    scipy.ndimage       import rotate
#from    astropy.cosmology   import Planck18
#import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm.auto                import tqdm
from concurrent.futures     import ThreadPoolExecutor, as_completed
import sys, os
from IPython.display import clear_output
from scripts.tools   import *
import re

obj_lis = Table.read('obj_lis_selected.fits')
print(len(obj_lis))

336


###  gen imput parameters for tinytim

In [2]:
def gen_psf_input_param(obj):
    try:
        col_names = ['identifier', 'rootname',  'pad', 'filter', 'pa' ,'focus', 'coord_ha', 'coord_hb', 'wavelen_ha', 'wavelen_hb','DELTATIM']
        beam_path = f"data_products/{file_name(obj,'beams')}"

                #chekc` if the file already exists`
        if os.path.exists(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits"):
            return f"{obj['subfield']}_{obj['id']} already exists"

        with fits.open(beam_path) as hdu:
            rows = []
            for image in hdu:
                if image.name == 'SCI':
                    #rootname, pad, shape filter
                    rootname = image.header['ROOTNAME']
                    identifier = f"{obj['subfield']}_{obj['id']}_{image.header['ROOTNAME']}"
                    pad        = image.header.get('PAD',0)
                    shape      = np.array(image.data.shape)
                    filter     = image.header['filter']

                    #thumbnail rel. pos
                    x0_crop = image.header['ORIGINX']
                    y0_crop = image.header['ORIGINY']

                    # this part need actual pixel coord for each Ha and Hb wavelen
                    coord_ha =  np.array((x0_crop,y0_crop)) - pad  #shape/2 +
                    coord_hb =  np.array((x0_crop,y0_crop)) - pad  #shape/2 

                    # this still needs focus information
                    focus = 0
                    #positon angle
                    pa = image.header['ORIENTAT']

                    #generate spectrum for halpha and hb
                    wavelen_ha = 656.28*(1+obj['z_MAP'])
                    wavelen_hb = 486.13*(1+obj['z_MAP'])

                    #integrationtime
                    deltatim = image.header['DELTATIM']

                    rows.append((identifier, rootname,  pad, filter, pa ,focus, coord_ha, coord_hb, wavelen_ha, wavelen_hb,deltatim))
            
            #save tiny tim & drizzle input param table
            Table(rows=rows,names=col_names).write(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits",overwrite=True)
            return f"{obj['subfield']}_{obj['id']} processed"
            
    except Exception as e:
        print(f"Error processing {obj['subfield']}_{obj['id']}: {e}")
        return f"!error {obj['subfield']}_{obj['id']}: {e}"


from concurrent.futures     import ThreadPoolExecutor, as_completed
def cat_process_gen_psf_input_param(obj_lis,max_threads=1):
    #make directories:
    os.makedirs('psf/obj_param/',exist_ok=True)
    results = []
    if max_threads>1:
        with ThreadPoolExecutor(max_threads) as executor:                                                       
            futures = {executor.submit(gen_psf_input_param,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        for obj in tqdm(obj_lis[obj_lis['manual_select']=='keep']):
            results.append(gen_psf_input_param(obj))
        return results


if __name__ == '__main__':
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process_gen_psf_input_param(obj_lis,max_threads=15)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)


Processing:   0%|          | 0/336 [00:00<?, ?it/s]

total number of obj processed: 336
number of failed obj 0


In [3]:
pas = np.array([])
filters = np.array([])
num_pas_per_exposure = np.array([])
num_exposure_per_obj = np.array([])

keep = obj_lis['manual_select']=='keep'
for obj in tqdm(obj_lis[keep]): 
    try:
        table  = Table.read(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits")
        pas    = np.append(pas,table['pa'])
        filters = np.append(filters,table['filter'])
        num_pas_per_exposure = np.append(num_pas_per_exposure,len(np.unique(np.round(table['pa'],decimals=1))))
        num_exposure_per_obj = np.append(num_exposure_per_obj,len(table))
    except Exception as e:
        print(f"Error reading {obj['subfield']}_{obj['id']}: {e}")
        continue

print('number of exposure per obj:',np.mean(num_exposure_per_obj))
print('number of pas per exposure:',np.mean(num_pas_per_exposure))
print('number of unique pas:',len(np.unique(np.round(pas,1))))
print('number of exposures per pa:')
g102 = filters==b'G102'
g141 = filters==b'G141'


print('average number of exposures in 102 grism',np.mean(np.unique(np.round(pas,1)[g102],return_counts=True)[1]))
print('average number of exposures in 141 grism',np.mean(np.unique(np.round(pas,1)[g141],return_counts=True)[1]))

  0%|          | 0/286 [00:00<?, ?it/s]

number of exposure per obj: 45.08041958041958
number of pas per exposure: 4.5524475524475525
number of unique pas: 65
number of exposures per pa:
average number of exposures in 102 grism 229.74468085106383
average number of exposures in 141 grism 104.75


### generate psf by calling tinytim + rotation and crop

In [4]:
def rotate_and_crop(image_path,rotating_angle=0,crop_size=30):
    with fits.open(image_path) as hdu:

        rot = rotate(hdu[0].data, 
                    angle=rotating_angle, 
                    reshape=False, 
                    mode='constant')
        
        center_x, center_y = np.array(hdu[0].data.shape) // 2
        half_crop = crop_size // 2
        cropped_data = rot[
            center_y - half_crop : center_y + half_crop,
            center_x - half_crop : center_x + half_crop]
        image = fits.ImageHDU(data=cropped_data, header=hdu[0].header)
    return image

#using tinytim and rotate to generate single psf
def gen_psf(obj,exist_skip=False):
    try:
        #path to save individual psf and combined psf
        save_path          = f"psf/individual_psf/{obj['subfield']}_{obj['id']}"
        save_path_combined = f"psf/combined_psf/{obj['subfield']}_{obj['id']}"

        #check if files already exist
        if os.path.exists(f"{save_path}_psf.fits") and os.path.exists(f"{save_path_combined}_ha.fits") and os.path.exists(f"{save_path_combined}_hb.fits"):
            clear_output(wait=True)
            return f"{obj['subfield']}_{obj['id']} already exists"
        else:
            print(f"Processing {obj['subfield']}_{obj['id']}")
        #load parameter table
        param_table = Table.read(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits")
        #use integration time as weight
        int_time = np.array(param_table['DELTATIM'])
        weights = int_time/np.sum(int_time)
        #psf_lis to contain individual psf
        psf_lis = fits.HDUList(); psf_lis.append(fits.PrimaryHDU())
        #_combine to store weighted average
        ha_combine = fits.ImageHDU(data=np.zeros((30,30)));hb_combine = fits.ImageHDU(data=np.zeros((30,30)))
        
        #calculate for each individual beam file:
        for i,row in tqdm(enumerate(param_table)):
            print(row)
            identifier, rootname, pad, filter, pa ,focus, coord_ha, coord_hb, wavelen_ha, wavelen_hb, deltatim = row

    #------------------this part generate and rotate individual psf--------------------------------
            #ha
            filename_ha = f"psf/individual_psf/{identifier.split('_')[2]}_ha.fits"
            tinytim.make_subsampled_model_psf(filename=filename_ha,
                                psf_size = 4,
                                filter_name = filter,
                                focus = focus,  
                                psf_position = coord_ha,
                                mono = wavelen_ha,
                                subsampling_factor = 2,
                                exist_skip=True)
            ha_psf = rotate_and_crop(filename_ha,-pa)
            ha_psf.name= f'{rootname}_ha'
            ha_combine.data += ha_psf.data * weights[i]
            
            #hb
            filename_hb = f"psf/individual_psf/{identifier.split('_')[2]}_hb.fits"
            tinytim.make_subsampled_model_psf(filename=filename_hb,
                                psf_size = 4,
                                filter_name = filter,
                                focus = focus,  
                                psf_position = coord_hb,
                                mono = wavelen_hb,
                                subsampling_factor = 2,
                                exist_skip=True)
            hb_psf = rotate_and_crop(filename_hb,-pa)
            hb_psf.name = f'{rootname}_hb'
            hb_combine.data += hb_psf.data * weights[i]
    #----------------------------------------------------------------------------------------------

            #normalization and append individual psf
            ha_psf.data = ha_psf.data/np.sum(ha_psf.data)
            hb_psf.data = hb_psf.data/np.sum(hb_psf.data)
            psf_lis.append(ha_psf);psf_lis.append(hb_psf)
            os.remove(filename_ha);os.remove(filename_hb)

        #save combined & individual psf
        psf_lis.append(ha_combine);psf_lis.append(hb_combine)
        psf_lis.writeto(f'{save_path}_psf.fits',overwrite=True)
        ha_combine.writeto(f'{save_path_combined}_ha.fits',overwrite=True)
        hb_combine.writeto(f'{save_path_combined}_hb.fits',overwrite=True)

        #clear output and return
        clear_output(wait=True)
        return f"{obj['subfield']}_{obj['id']} processed"

    except Exception as e:
        print(f"Error processing {obj['subfield']}_{obj['id']}: {e}")
        return f"! {obj['subfield']}-{obj['ID']} failed, error:{e}"



from concurrent.futures     import ThreadPoolExecutor, as_completed
def cat_process_gen_psf_from_input_param(obj_lis,max_threads=1):
    #make directories:
    os.makedirs('psf/individual_psf',exist_ok=True)
    os.makedirs('psf/combined_psf',exist_ok=True)  
    results = []
    if max_threads>1:
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(gen_psf,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        for obj in tqdm(obj_lis):
            results.append(gen_psf(obj))
        return results


if __name__ == '__main__':
    obj_lis = Table.read('obj_lis_selected.fits')
    #obj_lis = Table.read('failed_objects.fits')
    results = cat_process_gen_psf_from_input_param(obj_lis,max_threads=15)
    number = 0
    error_table = Table(names=['subfield', 'ID', 'error'], dtype=['str', 'str', 'str'])
    for result in results:
        if 'error' in result:
            number += 1
            # 用正则提取
            m = re.match(r"! ([^-]+)-([^\s]+) failed, error:(.*)", result)
            if m:
                subfield, id_, error = m.group(1), m.group(2), m.group(3)
            else:
                subfield, id_, error = '', '', result
            error_table.add_row([subfield, id_, error])
            print(result)
    error_table.write('failed_objects.fits', overwrite=True)

    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

! GN7-22746 failed, error:Format could not be identified based on the file name or contents, please provide a 'format' argument.
The available formats are:
           Format           Read Write Auto-identify Deprecated
--------------------------- ---- ----- ------------- ----------
                      ascii  Yes   Yes            No           
               ascii.aastex  Yes   Yes            No           
                ascii.basic  Yes   Yes            No           
                  ascii.cds  Yes    No            No           
     ascii.commented_header  Yes   Yes            No           
                  ascii.csv  Yes   Yes           Yes           
              ascii.daophot  Yes    No            No           
                 ascii.ecsv  Yes   Yes           Yes           
           ascii.fast_basic  Yes   Yes            No           
ascii.fast_commented_header  Yes   Yes            No           
             ascii.fast_csv  Yes   Yes            No           
       ascii

In [5]:
error_table = Table.read('failed_objects.fits')
error_table

subfield,ID,error
bytes3,bytes5,bytes102
GN7,22746,"Format could not be identified based on the file name or contents, please provide a 'format' argument."
